## Import libraries

In [1]:
import gc
import pickle
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

## Prepare data for model training

In [2]:
with open("../input/tps-may-data-preprocess/TPS_May_Dataset_w_Quantile.txt", 'rb') as handle: 
    data = handle.read()

processed_data = pickle.loads(data)
train_df = processed_data['train_df']
test_df = processed_data['test_df']

del processed_data
gc.collect()

0

In [3]:
cat_cols = ['feature_0','feature_2','feature_5','feature_9','feature_10','feature_11',
            'feature_12','feature_13','feature_17','feature_18','feature_22',
            'feature_29','feature_36','feature_37','feature_44']

train_df[cat_cols] = train_df[cat_cols].astype(int)
test_df[cat_cols] = test_df[cat_cols].astype(int)
cat_cols_indices = [train_df.columns.get_loc(col) for col in cat_cols]
print(cat_cols_indices)

[0, 2, 5, 9, 10, 11, 12, 13, 17, 18, 22, 29, 36, 37, 44]


In [4]:
Xtrain = train_df.loc[:, train_df.columns != 'target'].values
Ytrain = train_df['target'].values
Ytrain_oh = pd.get_dummies(train_df['target']).values
Xtest = test_df.values

print("Xtrain: {} \nYtrain: {} \nYtrain_oh: {} \nXtest: {}".format(Xtrain.shape, Ytrain.shape, 
                                                                   Ytrain_oh.shape, Xtest.shape))

del train_df
del test_df
gc.collect()

Xtrain: (99918, 951) 
Ytrain: (99918,) 
Ytrain_oh: (99918, 4) 
Xtest: (50000, 951)


20

## Build and validate the model

In [5]:
params = {}
params["objective"] = 'multiclass'
params['metric'] = 'multi_logloss'
params['boosting'] = 'gbdt'
#params["device_type"] = 'gpu'
params['num_class'] = 4
params['is_unbalance'] = True
params["learning_rate"] = 0.02
params["lambda_l2"] = 0.0256
params["num_leaves"] = 52
#params["max_depth"] = 10
params["feature_fraction"] = 0.503
params["bagging_fraction"] = 0.741
params["bagging_freq"] = 12
params["bagging_seed"] = 10
params["min_data_in_leaf"] = 10
params["verbosity"] = -1
num_rounds = 5000

In [6]:
FOLD = 10
NUM_SEED = 3

# Prediction Clipping Thresholds
p_min = 0.025
p_max = 1 - p_min

np.random.seed(3)
seeds = np.random.randint(0, 100, size=NUM_SEED)

oof_score = 0
y_pred_meta_lgb = np.zeros((Ytrain.shape[0], 4))
y_pred_final_lgb = np.zeros((Xtest.shape[0], 4))
counter = 0


for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain, Ytrain)):
        counter += 1

        train_x, train_y, train_y_oh = Xtrain[train], Ytrain[train], Ytrain_oh[train]
        val_x, val_y, val_y_oh = Xtrain[val], Ytrain[val], Ytrain_oh[val]

        lgtrain = lgb.Dataset(train_x, label=train_y.ravel())
        lgvalidation = lgb.Dataset(val_x, label=val_y.ravel())

        model = lgb.train(params, lgtrain, num_rounds, 
                          valid_sets=[lgtrain, lgvalidation], 
                          categorical_feature=cat_cols_indices,
                          early_stopping_rounds=200, verbose_eval=100)

        y_pred = model.predict(val_x, num_iteration=model.best_iteration)
        y_pred = np.clip(y_pred, p_min, p_max)
        y_pred_meta_lgb[val] += y_pred
        y_pred_final_lgb += model.predict(Xtest, num_iteration=model.best_iteration)
        
        score = log_loss(val_y_oh, y_pred)
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_lgb = y_pred_meta_lgb / float(NUM_SEED)
y_pred_final_lgb = y_pred_final_lgb / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 2, 5, 9, 10, 11, 12, 13, 17, 18, 22, 29, 36, 37, 44]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[100]	training's multi_logloss: 1.07308	valid_1's multi_logloss: 1.09851
[200]	training's multi_logloss: 1.04479	valid_1's multi_logloss: 1.09374
[300]	training's multi_logloss: 1.02227	valid_1's multi_logloss: 1.09248
[400]	training's multi_logloss: 1.00283	valid_1's multi_logloss: 1.09263
[500]	training's multi_logloss: 0.985113	valid_1's multi_logloss: 1.0931
Early stopping, best iteration is:
[306]	training's multi_logloss: 1.02101	valid_1's multi_logloss: 1.09239
Seed-24 | Fold-0 | OOF Score: 1.0923912573461216


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 2, 5, 9, 10, 11, 12, 13, 17, 18, 22, 29, 36, 37, 44]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[100]	training's multi_logloss: 1.07299	valid_1's multi_logloss: 1.09891
[200]	training's multi_logloss: 1.04471	valid_1's multi_logloss: 1.09334
[300]	training's multi_logloss: 1.02221	valid_1's multi_logloss: 1.09152
[400]	training's multi_logloss: 1.00258	valid_1's multi_logloss: 1.09146
[500]	training's multi_logloss: 0.985229	valid_1's multi_logloss: 1.09151
[600]	training's multi_logloss: 0.968644	valid_1's multi_logloss: 1.09131
[700]	training's multi_logloss: 0.953276	valid_1's multi_logloss: 1.09196
Early stopping, best iteration is:
[546]	training's multi_logloss: 0.977495	valid_1's multi_logloss: 1.09118
Seed-24 | Fold-1 | OOF Score: 1.0911784213439766


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 2, 5, 9, 10, 11, 12, 13, 17, 18, 22, 29, 36, 37, 44]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


...

Training until validation scores don't improve for 200 rounds
[100]	training's multi_logloss: 1.0728	valid_1's multi_logloss: 1.10026
[200]	training's multi_logloss: 1.04465	valid_1's multi_logloss: 1.09497
[300]	training's multi_logloss: 1.02221	valid_1's multi_logloss: 1.09294
[400]	training's multi_logloss: 1.0027	valid_1's multi_logloss: 1.09206
[500]	training's multi_logloss: 0.985155	valid_1's multi_logloss: 1.09238
[600]	training's multi_logloss: 0.968801	valid_1's multi_logloss: 1.09266
Early stopping, best iteration is:
[455]	training's multi_logloss: 0.992901	valid_1's multi_logloss: 1.09186
Seed-56 | Fold-7 | OOF Score: 1.0918572694985857


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 2, 5, 9, 10, 11, 12, 13, 17, 18, 22, 29, 36, 37, 44]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[100]	training's multi_logloss: 1.07317	valid_1's multi_logloss: 1.09872
[200]	training's multi_logloss: 1.0447	valid_1's multi_logloss: 1.09325
[300]	training's multi_logloss: 1.02209	valid_1's multi_logloss: 1.09192
[400]	training's multi_logloss: 1.00256	valid_1's multi_logloss: 1.09194
[500]	training's multi_logloss: 0.98487	valid_1's multi_logloss: 1.09224
Early stopping, best iteration is:
[344]	training's multi_logloss: 1.01322	valid_1's multi_logloss: 1.09173
Seed-56 | Fold-8 | OOF Score: 1.0917323486550343


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 2, 5, 9, 10, 11, 12, 13, 17, 18, 22, 29, 36, 37, 44]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 200 rounds
[100]	training's multi_logloss: 1.07259	valid_1's multi_logloss: 1.10132
[200]	training's multi_logloss: 1.04436	valid_1's multi_logloss: 1.09711
[300]	training's multi_logloss: 1.02194	valid_1's multi_logloss: 1.09619
[400]	training's multi_logloss: 1.00225	valid_1's multi_logloss: 1.09661
Early stopping, best iteration is:
[292]	training's multi_logloss: 1.02357	valid_1's multi_logloss: 1.09611
Seed-56 | Fold-9 | OOF Score: 1.0961073562129975

Seed: 56 | Aggregate OOF Score: 1.0924373886355947


Aggregate OOF Score: 1.0926958774050042


In [7]:
np.savez_compressed('./LGB_Meta_Features.npz',
                    y_pred_meta_lgb=y_pred_meta_lgb, 
                    oof_score=oof_score,
                    y_pred_final_lgb=y_pred_final_lgb)

## Create submission file

In [8]:
y_pred_final_lgb = np.clip(y_pred_final_lgb, p_min, p_max)

test_df = pd.read_csv("../input/tabular-playground-series-may-2021/test.csv")
submit_df = pd.DataFrame()
submit_df['id'] = test_df['id']
submit_df['Class_1'] = y_pred_final_lgb[:,0]
submit_df['Class_2'] = y_pred_final_lgb[:,1]
submit_df['Class_3'] = y_pred_final_lgb[:,2]
submit_df['Class_4'] = y_pred_final_lgb[:,3]
submit_df.head()

,id,Class_1,Class_2,Class_3,Class_4
0,100000,0.084972,0.607259,0.185489,0.122280
1,100001,0.077911,0.668733,0.173727,0.079629
2,100002,0.085170,0.637511,0.175553,0.101767
3,100003,0.079365,0.540842,0.265400,0.114392
4,100004,0.076234,0.608782,0.211749,0.103235


In [9]:
submit_df.to_csv("./LGB_submission.csv", index=False)